<h1>Segmenting and clustering in Toronto´s neighborhood-part3</h1>

<h2>1. Set the data</h2>

Import the libraries needed

In [1]:
from geopy.geocoders import Nominatim
import folium
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [18]:
address = 'El Retablo, Comas'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geographical coordinate of El Retablo, Comas are -11.9382646, -77.0600983.


In [3]:
import urllib.request

In [21]:
# create map of Toronto using latitude and longitude values
map_Lima = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=address,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Lima) 
map_Lima

In [5]:
# -----------------------------------------------------------------------------
# Name:        html_table_parser
# Purpose:     Simple class for parsing an (x)html string to extract tables.
#              Written in python3
#
# Author:      Josua Schmid
#
# Created:     05.03.2014
# Copyright:   (c) Josua Schmid 2014
# Licence:     AGPLv3
# -----------------------------------------------------------------------------

from html.parser import HTMLParser


class HTMLTableParser(HTMLParser):
    """ This class serves as a html table parser. It is able to parse multiple
    tables which you feed in. You can access the result per .tables field.
    """
    def __init__(
        self,
        decode_html_entities=False,
        data_separator=' ',
    ):

        HTMLParser.__init__(self, convert_charrefs=decode_html_entities)

        self._data_separator = data_separator

        self._in_td = False
        self._in_th = False
        self._current_table = []
        self._current_row = []
        self._current_cell = []
        self.tables = []

    def handle_starttag(self, tag, attrs):
        """ We need to remember the opening point for the content of interest.
        The other tags (<table>, <tr>) are only handled at the closing point.
        """
        if tag == 'td':
            self._in_td = True
        if tag == 'th':
            self._in_th = True

    def handle_data(self, data):
        """ This is where we save content to a cell """
        if self._in_td or self._in_th:
            self._current_cell.append(data.strip())
    
    def handle_endtag(self, tag):
        """ Here we exit the tags. If the closing tag is </tr>, we know that we
        can save our currently parsed cells to the current table as a row and
        prepare for a new row. If the closing tag is </table>, we save the
        current table and prepare for a new one.
        """
        if tag == 'td':
            self._in_td = False
        elif tag == 'th':
            self._in_th = False

        if tag in ['td', 'th']:
            final_cell = self._data_separator.join(self._current_cell).strip()
            self._current_row.append(final_cell)
            self._current_cell = []
        elif tag == 'tr':
            self._current_table.append(self._current_row)
            self._current_row = []
        elif tag == 'table':
            self.tables.append(self._current_table)
            self._current_table = []

In [6]:
#Target the URL of Wikipedia
target='https://en.wikipedia.org/wiki/List_of_districts_of_Lima'

#Request method to find URL and get que website content
req=urllib.request.Request(url=target)
f=urllib.request.urlopen(req)
xhtml=f.read().decode('utf-8')

#Instantiate HTMLTableParser Class and feed it
p=HTMLTableParser()
p.feed(xhtml)

#Write the table into Python list
Table_list=p.tables

 #Use from_dict to method  to read the table into a Pandas dataframe
Lima_df=pd.DataFrame.from_dict(Table_list[0])

In [7]:
Lima_df.head()

,0,1,2,3,4,5,6,7
0,Districts,UBIGEO,Area (km²),Population,Population density (/km²),Created,Postal code,Location
1,Ancón,150102,299.22,"29,419",98.3,18741029 29 October 1874,02,
2,Ate,150103,77.72,"419,663","5,399.7",18570102 2 January 1857,03,
3,Barranco,150104,3.33,"45,922","13,790.4",18741026 26 October 1874,04,
4,Breña,150105,3.22,"94,808","29,443.5",19490715 15 July 1949,05,


In [8]:
Lima_df.rename(columns=Lima_df.iloc[0],inplace=True)
Lima_df.drop([0], inplace=True)

In [9]:
Lima_df.drop(columns='Location', inplace=True)

In [10]:
Lima_df.head()

,Districts,UBIGEO,Area (km²),Population,Population density (/km²),Created,Postal code
1,Ancón,150102,299.22,"29,419",98.3,18741029 29 October 1874,02
2,Ate,150103,77.72,"419,663","5,399.7",18570102 2 January 1857,03
3,Barranco,150104,3.33,"45,922","13,790.4",18741026 26 October 1874,04
4,Breña,150105,3.22,"94,808","29,443.5",19490715 15 July 1949,05
5,Carabayllo,150106,346.88,"188,764",544.2,18210804 4 August 1821,06


In [11]:
Lima_df=Lima_df.reset_index()
Lima_df.drop(['index'], axis=1, inplace=True)
Lima_df.head()

,Districts,UBIGEO,Area (km²),Population,Population density (/km²),Created,Postal code
0,Ancón,150102,299.22,"29,419",98.3,18741029 29 October 1874,02
1,Ate,150103,77.72,"419,663","5,399.7",18570102 2 January 1857,03
2,Barranco,150104,3.33,"45,922","13,790.4",18741026 26 October 1874,04
3,Breña,150105,3.22,"94,808","29,443.5",19490715 15 July 1949,05
4,Carabayllo,150106,346.88,"188,764",544.2,18210804 4 August 1821,06


In [12]:
Lima_df=Lima_df.replace({'Santa María del Mar District':'Santa María del Mar'})

In [13]:
Lima_df.head()

,Districts,UBIGEO,Area (km²),Population,Population density (/km²),Created,Postal code
0,Ancón,150102,299.22,"29,419",98.3,18741029 29 October 1874,02
1,Ate,150103,77.72,"419,663","5,399.7",18570102 2 January 1857,03
2,Barranco,150104,3.33,"45,922","13,790.4",18741026 26 October 1874,04
3,Breña,150105,3.22,"94,808","29,443.5",19490715 15 July 1949,05
4,Carabayllo,150106,346.88,"188,764",544.2,18210804 4 August 1821,06


In [14]:
address = Lima_df['Districts']
latitud=[]
longitud=[]
for i in address:
    geolocator = Nominatim(user_agent="tor_explorer")
    location = geolocator.geocode('{}, Perú'.format(i))
    latitude = location.latitude
    latitud.append(latitude)
    longitude = location.longitude
    longitud.append(longitude)
    #print('The geographical coordinate of {} City are {}, {}.'.format(i,latitude, longitude))
d={'Districts':Lima_df['Districts'],'Latitud':latitud,'Longitud':longitud}
cordinates=pd.DataFrame(d)
cordinates.head()

,Districts,Latitud,Longitud
0,Ancón,-11.696554,-77.111655
1,Ate,-12.038728,-76.896873
2,Barranco,-12.143959,-77.020268
3,Breña,-12.059700,-77.050119
4,Carabayllo,-11.794993,-76.989292


In [15]:
lima_merge=pd.merge(Lima_df,cordinates)

In [41]:
lima_merge

,Districts,UBIGEO,Area (km²),Population,Population density (/km²),Created,Postal code,Latitud,Longitud
0,Ancón,150102,299.22,"29,419",98.3,18741029 29 October 1874,02,-11.696554,-77.111655
1,Ate,150103,77.72,"419,663","5,399.7",18570102 2 January 1857,03,-12.038728,-76.896873
2,Barranco,150104,3.33,"45,922","13,790.4",18741026 26 October 1874,04,-12.143959,-77.020268
3,Breña,150105,3.22,"94,808","29,443.5",19490715 15 July 1949,05,-12.059700,-77.050119
4,Carabayllo,150106,346.88,"188,764",544.2,18210804 4 August 1821,06,-11.794993,-76.989292
5,Chaclacayo,150107,39.5,"39,686","1,004.7",19400424 24 April 1940,08,-11.975740,-76.769871
6,Chorrillos,150108,38.94,"262,595","6,743.6",18570102 2 January 1857,09,-12.192350,-77.008962
7,Cieneguilla,150109,240.33,"15,784",65.7,19700303 3 March 1970,40,-12.073167,-76.777071
8,Comas,150110,48.75,"464,745","9,533.2",19611212 12 December 1961,07,-11.932861,-77.040674
9,El Agustino,150111,12.54,"165,425","13,191.8",19650106 6 January 1965,10,-12.042052,-76.995714


In [17]:
# create map of Toronto using latitude and longitude values
map_Lima = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers to map
for lat, lng, dist in zip(lima_merge['Latitud'], lima_merge['Longitud'], lima_merge['Districts']):
    label = '{}, Lima'.format(dist)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Lima)  
    
map_Lima

Create a folium maps to display the neighborhood from the table df

<h2>2. Define the Foursquare credentials and version</h2>

In [22]:
CLIENT_ID = 'BZUPJWEGWZ3B3QSCAMXPKW4IWNBBTIP1BNUPNAK2GIKVWCPW' # your Foursquare ID
CLIENT_SECRET = '1C4FQNIT0HCZHXPEPBM3QXRQYFUYZ0LAWKYSNCJ1RCPMC0OL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BZUPJWEGWZ3B3QSCAMXPKW4IWNBBTIP1BNUPNAK2GIKVWCPW
CLIENT_SECRET:1C4FQNIT0HCZHXPEPBM3QXRQYFUYZ0LAWKYSNCJ1RCPMC0OL


<h2>2. Preparing & cleaning the data</h2>

In [49]:
neighborhood_latitude = lima_merge.loc[2, 'Latitud'] # neighborhood latitude value
neighborhood_longitude = lima_merge.loc[2, 'Longitud'] # neighborhood longitude value

neighborhood_name = lima_merge.loc[2, 'Districts'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Barranco are -12.143959, -77.02026805535999.


In [50]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=BZUPJWEGWZ3B3QSCAMXPKW4IWNBBTIP1BNUPNAK2GIKVWCPW&client_secret=1C4FQNIT0HCZHXPEPBM3QXRQYFUYZ0LAWKYSNCJ1RCPMC0OL&v=20180605&ll=-12.143959,-77.02026805535999&radius=500&limit=100'

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f41441ebe0e0c62272d87c3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Barranco',
  'headerFullLocation': 'Barranco, Lima',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 111,
  'suggestedBounds': {'ne': {'lat': -12.139458995499997,
    'lng': -77.01567363961514},
   'sw': {'lat': -12.148459004500005, 'lng': -77.02486247110484}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5884509b4bafb766c20be39d',
       'name': 'FUGA',
       'location': {'address': 'Grau 701 Barranco',
        'lat': -12.144419289920483,
        'lng': -77.02200511063033,
        'labeledLatLngs': [{'label': 'display',
          'lat': -12.14441928

In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [53]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,FUGA,Nightclub,-12.144419,-77.022005
1,Pizerrìa La Linterna,Pizza Place,-12.144713,-77.021933
2,Sargento Pimienta,Bar,-12.143782,-77.018755
3,Amoramar,Seafood Restaurant,-12.144075,-77.018201
4,Hotel B,Hotel,-12.142954,-77.023266


In [54]:
nearby_venues

,name,categories,lat,lng
0,FUGA,Nightclub,-12.144419,-77.022005
1,Pizerrìa La Linterna,Pizza Place,-12.144713,-77.021933
2,Sargento Pimienta,Bar,-12.143782,-77.018755
3,Amoramar,Seafood Restaurant,-12.144075,-77.018201
4,Hotel B,Hotel,-12.142954,-77.023266
5,DaDA,Bar,-12.145049,-77.022843
6,Belgicius,Belgian Restaurant,-12.141295,-77.021437
7,La Espiga de Oro,Bakery,-12.142968,-77.022081
8,Sandwiches Monstruos,Sandwich Place,-12.141153,-77.021764
9,Canta Ranita,Seafood Restaurant,-12.146824,-77.021059


In [55]:
# one hot encoding
barranco_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
barranco_onehot['name'] = nearby_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [barranco_onehot.columns[-1]] + list(barranco_onehot.columns[:-1])
barranco_onehot = barranco_onehot[fixed_columns]

barranco_onehot.head()

,name,Arepa Restaurant,Art Gallery,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Bed & Breakfast,Belgian Restaurant,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Café,Chinese Restaurant,Coffee Shop,Comedy Club,Concert Hall,Deli / Bodega,Falafel Restaurant,French Restaurant,Fried Chicken Joint,Gastropub,Health Food Store,Hostel,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Lounge,Market,Mediterranean Restaurant,Nail Salon,Nightclub,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pizza Place,Plaza,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Spanish Restaurant,Speakeasy,Steakhouse,Sushi Restaurant,Theater,Wine Bar
0,FUGA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Pizerrìa La Linterna,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Sargento Pimienta,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Amoramar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Hotel B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Defining funtion to get a nearby venues

In [79]:
len(barranco_onehot.columns)

53

In [80]:
barranco_onehot.shape

(100, 53)

In [94]:
filas=len(barranco_onehot.index)
columnas=len(barranco_onehot.columns)
suma=[]
for j in range(columnas-1):
    total=sum(barranco_onehot.iloc[0:,j+1])
    suma.append(total)


In [97]:
len(barranco_onehot.index)

100

In [73]:
sum(barranco_onehot.iloc[0:,52])

1

In [92]:
range(len(barranco_onehot.columns))

range(0, 53)

In [103]:
barranco_onehot.describe()

,Arepa Restaurant,Art Gallery,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Bed & Breakfast,Belgian Restaurant,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Café,Chinese Restaurant,Coffee Shop,Comedy Club,Concert Hall,Deli / Bodega,Falafel Restaurant,French Restaurant,Fried Chicken Joint,Gastropub,Health Food Store,Hostel,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Lounge,Market,Mediterranean Restaurant,Nail Salon,Nightclub,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pizza Place,Plaza,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Spanish Restaurant,Speakeasy,Steakhouse,Sushi Restaurant,Theater,Wine Bar
count,100.00,100.000000,100.00,100.00,100.000000,100.000000,100.00,100.00,100.000000,100.00,100.000000,100.00,100.000000,100.000000,100.000000,100.000000,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.000000,100.000000,100.000000,100.00,100.00,100.00,100.00,100.00,100.000000,100.00,100.00,100.00,100.000000,100.00,100.000000,100.000000,100.00,100.00,100.000000,100.000000,100.000000,100.00,100.00,100.00,100.00,100.000000,100.00
mean,0.01,0.030000,0.01,0.01,0.030000,0.110000,0.01,0.01,0.020000,0.01,0.040000,0.01,0.050000,0.030000,0.020000,0.020000,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.020000,0.020000,0.030000,0.01,0.01,0.01,0.01,0.01,0.020000,0.01,0.01,0.01,0.020000,0.01,0.030000,0.030000,0.01,0.01,0.040000,0.040000,0.060000,0.01,0.01,0.01,0.01,0.020000,0.01
std,0.10,0.171447,0.10,0.10,0.171447,0.314466,0.10,0.10,0.140705,0.10,0.196946,0.10,0.219043,0.171447,0.140705,0.140705,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.140705,0.140705,0.171447,0.10,0.10,0.10,0.10,0.10,0.140705,0.10,0.10,0.10,0.140705,0.10,0.171447,0.171447,0.10,0.10,0.196946,0.196946,0.238683,0.10,0.10,0.10,0.10,0.140705,0.10
min,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
25%,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
50%,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
75%,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
max,1.00,1.000000,1.00,1.00,1.000000,1.000000,1.00,1.00,1.000000,1.00,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00,1.00,1.000000,1.00,1.00,1.00,1.000000,1.00,1.000000,1.000000,1.00,1.00,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00,1.000000,1.00


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the code above to create a new dataframe called *toronto_venue*

In [ ]:
LIMIT = 100
radius = 500
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood_name'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [ ]:
toronto_venues.shape

Lets check how many unique venues category were return for each neigborhoood

In [ ]:
print('Hay {} categorías'.format(len(toronto_venues['Venue Category'].unique())))

<h2>3. Analyze each neighborhood</h2>

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood_name'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Lets group row by Neighborhood to calculated the mean of the frecuency of ocurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood_name').mean().reset_index()
toronto_grouped.head()

In [ ]:
toronto_grouped.shape

Lets print the top 5 most commun venue for each Neighborhood

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood_name']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood_name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Lets create a dataframe called **neighborhoods_venues_sorted**

In [ ]:
num_top_venues = 10
cardinals = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood_name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, cardinals[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood_name'] = toronto_grouped['Neighborhood_name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<h2>4. Cluster Neighborhoods</h2>

Run k-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood_name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:38] 

Notice the result, is an array with the cluster number for each row

Now, lets create a new label called *'Cluster labels'* with the **kmeans.label_** values

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Merge two dataframes, *toronto_data* and *neighborhoods_venues_sorted*

In [ ]:
toronto_merged=pd.merge(toronto_data,neighborhoods_venues_sorted)

Lets convert the data type of the column *Cluster Labels* to int to make sure that there is no error.

In [ ]:
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)

In [ ]:
toronto_merged

Finally lets create a map and markers the cluster

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood_name'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters